In [ ]:
from core.meta import meta_interface
%matplotlib widget

In [ ]:
d = meta_interface(project_config_filepath="test_data/project_config.yaml", project_name="test_data")

In [ ]:
d.add_recording_config("/Users/kobel/Downloads/220922/recording_config_220922.yaml")

In [ ]:
d.initialize_meta_config()

In [ ]:
d.create_recordings()

In [ ]:
d.synchronize_recordings(verbose = True, test_mode = True)  # test_mode: bool, doesnt overwrite synchronized and analysed videos
# exclude recordingvideos based on synchronization

In [ ]:
d.create_calibrations(ground_truth_config_filepath="test_data/ground_truth_config.yaml")

In [ ]:
d.synchronize_calibrations(test_mode = True)  

In [ ]:
d.calibrate(verbose = False)

In [ ]:
d.triangulate_recordings()

In [ ]:
d.export_meta_to_yaml(filepath="test_data/My_project.yaml")

In [ ]:
d.load_meta_from_yaml(filepath="test_data/test_data_01.yaml")

In [ ]:
d.objects['position_objects']['220922_0_Bottom_Ground1_Ground2_Side1_Side2_Side3_Top']

In [ ]:
from pathlib import Path
from core.triangulation_calibration_module import (
    Calibration,
    Triangulation_Positions,
    Triangulation_Recordings,
)

In [ ]:
rec_config = Path("/Users/kobel/Downloads/220922/recording_config_220922.yaml")
calibration_object = Calibration(calibration_directory=rec_config.parent, 
                                 output_directory=rec_config.parent, 
                                 recording_config_filepath=rec_config, 
                                 project_config_filepath = "test_data/project_config.yaml")

In [ ]:
calibration_object.run_synchronization(test_mode = True)

In [ ]:
calibration_object.run_calibration(verbose=True)

In [ ]:
rec_config = Path("/Users/kobel/Downloads/220927/recording_config_270922.yaml")
calibration_object = Calibration(calibration_directory=rec_config.parent, 
                                 output_directory=rec_config.parent, 
                                 recording_config_filepath=rec_config, 
                                 project_config_filepath = "test_data/project_config.yaml")
calibration_object.run_synchronization(test_mode = True)

In [ ]:
calibration_object.run_calibration(verbose=True)

In [ ]:
21.09. Side3 to be excluded! do calibration!
22.09. 
23.09. Top to be excluded!
27.09.
19.12. error 0.95
20.12. Side3 not well synchronized, exclude! redo calibration!
21.12. Side2 and Side3 not well synchronized, exclude! redo calibration! Top has 15000 frames less (why?, how?)
22.12. error 2.22 Side3 not well synchronized, exclude! redo calibration!
# calibration error wenn eine Kamera 0 boards detektiert hat!

In [ ]:
a = Calibration(
    calibration_directory="test_data/Server_structure/Calibrations/220826/",
    project_config_filepath="test_data/project_config.yaml",
    recording_config_filepath="test_data/Server_structure/Calibrations/220826/recording_config.yaml",
    output_directory="test_data/Server_structure/Calibrations/220826/",
)

In [ ]:
a.run_synchronization()

In [ ]:
a.run_calibration(verbose=True)

In [ ]:
b = Triangulation_Recordings(
    recording_directory="test_data/Server_structure/VGlut2-flp/September2022/196_F7-27/220826_OTT/",
    calibration_directory="test_data/Server_structure/Calibrations/220826/",
    recording_config_filepath="test_data/Server_structure/Calibrations/220826/recording_config.yaml",
    project_config_filepath="test_data/project_config.yaml",
    output_directory="test_data/Server_structure/VGlut2-flp/September2022/196_F7-27/220826_OTT/",
)

In [ ]:
b.run_synchronization()

In [ ]:
b.run_triangulation(
    calibration_toml_filepath="test_data/Server_structure/Calibrations/220826/calibration_220826.toml"
)

In [ ]:
c = Triangulation_Positions(
    positions_directory="test_data/Server_structure/Calibrations/220826/",
    calibration_directory="test_data/Server_structure/Calibrations/220826/",
    recording_config_filepath="test_data/Server_structure/Calibrations/220826/recording_config.yaml",
    project_config_filepath="test_data/project_config.yaml",
    output_directory="test_data/Server_structure/Calibrations/220826/",
)

In [ ]:
c.get_marker_predictions()

In [ ]:
c.run_triangulation(
    calibration_toml_filepath="test_data/Server_structure/Calibrations/220826/calibration_220826.toml"
)

In [ ]:
# special dependencies

In [ ]:
"""
pip install aniposelib
pip install ffmpeg
pip install ffmpeg-python
pip install imageio-ffmpeg
jupyter lab
Tkinter
"""